# Plant Pathology 2020 - FGVC7 | MLE-Benchmark Medal Plan

Objective: Win a medal by maximizing mean-column-wise-roc-auc. Default target hint says `multiple_diseases`, but the original competition is multi-label with columns: healthy, multiple_diseases, rust, scab. I will explore train.csv to confirm targets and shape approach accordingly.

Plan:
- Data audit: load CSVs, verify columns, image paths, counts, label distributions; sanity-check image readability.
- Baseline CV: Stratified KFold (multilabel stratification if multi-label; else standard stratify) with strong image model.
- Modeling v1: pretrained CNN (timm, e.g., tf_efficientnet_b0/b3) with 384px, BCEWithLogits loss, Mixup/CutMix off initially, simple augmentations.
- Training details: 5 folds, 2-stage training (256px warmup then 384px), early stopping, cosine LR, weight decay; log fold times.
- Ensembling: if time permits, average 2 backbones (b0 + b3) or 2 seeds.
- Submission: out-of-fold AUC tracking; generate test preds and submission.csv.

Experiment Log:
- [ ] E0: Data loading + EDA
- [ ] E1: EfficientNet-b0 256px 5-fold baseline
- [ ] E2: Resize 384px finetune, TTA
- [ ] E3: Upgrade to b3/b4 if time allows
- [ ] E4: Seed ensemble / test-time augmentations

Checkpoints: I will request expert review after plan, after EDA, after first baseline CV, and if scores stall.

In [1]:
# E0: Data loading + quick audit
import os, sys, time, json, math, gc, random
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path('.')
IMG_DIR = DATA_DIR / 'images'

t0 = time.time()
print('CWD:', os.getcwd())
print('Images dir exists:', IMG_DIR.exists())
print('Num images:', len(list(IMG_DIR.glob('*.jpg'))))

train = pd.read_csv(DATA_DIR / 'train.csv')
test = pd.read_csv(DATA_DIR / 'test.csv')
print('train.shape:', train.shape, 'test.shape:', test.shape)
print('train.columns:', train.columns.tolist())
print('test.columns:', test.columns.tolist())
print('\ntrain.head():')
display(train.head())

# Identify target columns (original competition has 4 one-hot label columns)
potential_targets = ['healthy','multiple_diseases','rust','scab']
target_cols = [c for c in potential_targets if c in train.columns]
if len(target_cols) == 0 and 'multiple_diseases' in train.columns:
    target_cols = ['multiple_diseases']
print('Detected target columns:', target_cols)

# Identify image id column
img_col_candidates = [c for c in ['image_id','image','id','filename'] if c in train.columns]
img_col = img_col_candidates[0] if img_col_candidates else None
print('Image column:', img_col)

# Check image file existence for a few samples
def img_path_from_row(row):
    if img_col is None:
        return None
    # common pattern: {image_id}.jpg
    name = str(row[img_col])
    p = IMG_DIR / f"{name}.jpg"
    if not p.exists():
        # try uppercase/lowercase differences or prefixes
        alt = list(IMG_DIR.glob(f"{name}*.jpg"))
        if len(alt):
            return alt[0]
    return p

exist_checks = []
for i, row in train.head(10).iterrows():
    p = img_path_from_row(row)
    exist_checks.append((row[img_col] if img_col else None, p.name if p else None, p.exists() if p else False))
print('Sample image existence checks (first 10):')
for rec in exist_checks:
    print(rec)

# Label distribution
if len(target_cols) > 1:
    print('\nLabel sums (multi-label one-hot):')
    print(train[target_cols].sum())
else:
    # if single target provided as numeric/class, show value_counts
    print('\nSingle target distribution:')
    print(train[target_cols[0]].value_counts(dropna=False))

print(f'Elapsed: {time.time()-t0:.2f}s')

CWD: /app/agent_run_states/plant-pathology-2020-fgvc7-spray-20250908-012355
Images dir exists: True
Num images: 1821
train.shape: (1638, 5) test.shape: (183, 1)
train.columns: ['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab']
test.columns: ['image_id']

train.head():


,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,1,0
1,Train_1,1,0,0,0
2,Train_2,0,0,1,0
3,Train_3,1,0,0,0
4,Train_4,0,0,1,0


Detected target columns: ['healthy', 'multiple_diseases', 'rust', 'scab']
Image column: image_id
Sample image existence checks (first 10):
('Train_0', 'Train_0.jpg', True)
('Train_1', 'Train_1.jpg', True)
('Train_2', 'Train_2.jpg', True)
('Train_3', 'Train_3.jpg', True)
('Train_4', 'Train_4.jpg', True)
('Train_5', 'Train_5.jpg', True)
('Train_6', 'Train_6.jpg', True)
('Train_7', 'Train_7.jpg', True)
('Train_8', 'Train_8.jpg', True)
('Train_9', 'Train_9.jpg', True)

Label sums (multi-label one-hot):
healthy              468
multiple_diseases     85
rust                 551
scab                 534
dtype: int64
Elapsed: 0.02s


In [3]:
# Setup: installs, imports, seeds, utilities
import sys, subprocess, importlib, time, os, random

def ensure_package(pkg, import_name=None, version=None):
    import_name = import_name or pkg
    try:
        importlib.import_module(import_name)
        print(f"OK: {pkg} already installed")
    except Exception as e:
        pip_cmd = [sys.executable, '-m', 'pip', 'install', pkg + (f'=={version}' if version else '')]
        print('Installing', ' '.join(pip_cmd)); subprocess.check_call(pip_cmd)
        importlib.invalidate_caches()

# Core libs
ensure_package('timm')
ensure_package('albumentations')
ensure_package('iterative-stratification', 'iterstrat')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

SEED = 42
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
set_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# Metric helper
def mean_columnwise_roc_auc(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        try:
            s = roc_auc_score(y_true[:, i], y_pred[:, i])
        except ValueError:
            s = np.nan
        scores.append(s)
    return float(np.nanmean(scores)), scores

print('Setup complete.')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OK: timm already installed


OK: albumentations already installed
OK: iterative-stratification already installed


Device: cuda
Setup complete.


In [4]:
# Modeling pipeline: dataset, transforms, model, CV training/eval, inference
import os, time, math, gc, copy, warnings
from pathlib import Path

# Set writable cache dirs BEFORE importing timm/huggingface
CACHE_DIR = Path('./timm_cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)
os.environ['HF_HOME'] = str(CACHE_DIR)
os.environ['HF_HUB_CACHE'] = str(CACHE_DIR / 'hub')
os.environ['XDG_CACHE_HOME'] = str(CACHE_DIR)
os.environ['TIMM_HOME'] = str(CACHE_DIR)
os.environ['TORCH_HOME'] = str(CACHE_DIR / 'torch')
os.environ['PYTORCH_HUB'] = str(CACHE_DIR / 'pytorch_hub')

import numpy as np
import pandas as pd
import cv2
import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.metrics import roc_auc_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
print('Device:', device, flush=True)

warnings.filterwarnings('ignore')
IMG_DIR = Path('images')
TARGET_COLS = ['healthy','multiple_diseases','rust','scab']
N_CLASSES = len(TARGET_COLS)

# Transforms
IMG_SIZE = 384
def get_transforms(img_size=IMG_SIZE):
    train_tfms = A.Compose([
        A.RandomResizedCrop(size=(img_size, img_size), scale=(0.8, 1.0), ratio=(0.75, 1.333), p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=25, p=0.6),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.ColorJitter(p=0.3),
        A.CoarseDropout(max_holes=8, max_height=int(img_size*0.1), max_width=int(img_size*0.1), fill_value=0, p=0.5),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2(),
    ])
    valid_tfms = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2(),
    ])
    return train_tfms, valid_tfms

class PlantDataset(Dataset):
    def __init__(self, df, img_dir, img_col='image_id', target_cols=None, transforms=None, is_train=True):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.img_col = img_col
        self.target_cols = target_cols
        self.transforms = transforms
        self.is_train = is_train
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / f"{row[self.img_col]}.jpg"
        img = cv2.imread(str(img_path))
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.is_train and self.target_cols is not None:
            y = row[self.target_cols].values.astype('float32')
            return img, torch.from_numpy(y)
        else:
            return img, row[self.img_col]

# Model
class Model(nn.Module):
    def __init__(self, backbone='tf_efficientnet_b3_ns', num_classes=N_CLASSES, drop=0.2):
        super().__init__()
        # ensure pretrained weights cache in writable dir
        self.net = timm.create_model(backbone, pretrained=True, num_classes=num_classes, drop_rate=drop, in_chans=3, cache_dir=str(CACHE_DIR))
    def forward(self, x):
        return self.net(x)

# EMA utility
class EMA:
    def __init__(self, model, decay=0.9998):
        self.ema = copy.deepcopy(model).eval()
        for p in self.ema.parameters():
            p.requires_grad_(False)
        self.decay = decay
    @torch.no_grad()
    def update(self, model):
        msd = model.state_dict()
        for k, v in self.ema.state_dict().items():
            if k in msd:
                v.copy_(v * self.decay + msd[k] * (1.0 - self.decay))

# Asymmetric Loss for multi-label
class AsymmetricLossMultiLabel(nn.Module):
    def __init__(self, gamma_pos=0.0, gamma_neg=4.0, clip=0.05, eps=1e-8):
        super().__init__()
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.clip = clip
        self.eps = eps
    def forward(self, logits, targets):
        x_sigmoid = torch.sigmoid(logits)
        xs_pos = x_sigmoid
        xs_neg = 1.0 - x_sigmoid
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)
        # Basic CE
        loss = targets * torch.log(xs_pos.clamp(min=self.eps)) + (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        # Focusing
        if self.gamma_pos > 0 or self.gamma_neg > 0:
            pt = targets * xs_pos + (1 - targets) * xs_neg
            one_sided = (1 - pt) ** (self.gamma_pos * targets + self.gamma_neg * (1 - targets))
            loss *= one_sided
        return -loss.mean()

def compute_auc(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        try:
            scores.append(roc_auc_score(y_true[:, i], y_pred[:, i]))
        except Exception:
            scores.append(np.nan)
    return float(np.nanmean(scores)), scores

# Mixup
def do_mixup(images, targets, alpha=0.3):
    if alpha <= 0:
        return images, targets
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(images.size(0), device=images.device)
    mixed_images = lam * images + (1 - lam) * images[index]
    mixed_targets = lam * targets + (1 - lam) * targets[index]
    return mixed_images, mixed_targets

# Train/valid loops
def train_one_epoch(model, ema, loader, optimizer, scaler, loss_fn, device, scheduler=None, mixup_alpha=0.3, accum_steps=1):
    model.train()
    running = 0.0
    optimizer.zero_grad(set_to_none=True)
    for step, (images, targets) in enumerate(loader):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        if mixup_alpha and mixup_alpha > 0:
            images, targets = do_mixup(images, targets, alpha=mixup_alpha)
        with torch.cuda.amp.autocast(enabled=True):
            logits = model(images)
            loss = loss_fn(logits, targets) / accum_steps
        scaler.scale(loss).backward()
        # gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        if ((step + 1) % accum_steps) == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        if scheduler is not None:
            # OneCycleLR step per batch
            scheduler.step()
        if ema is not None:
            ema.update(model)
        running += loss.item() * accum_steps
        if (step+1) % 25 == 0:
            print(f"  step {step+1}/{len(loader)} loss {running/(step+1):.4f}", flush=True)
    return running / max(1, len(loader))

@torch.no_grad()
def validate(model, loader, device, use_ema=False, ema=None):
    model.eval()
    preds = []
    targs = []
    net = ema.ema if (use_ema and ema is not None) else model
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        logits = net(images)
        probs = torch.sigmoid(logits).float().cpu().numpy()
        preds.append(probs)
        targs.append(targets.numpy())
    preds = np.concatenate(preds, axis=0)
    targs = np.concatenate(targs, axis=0)
    mauc, per = compute_auc(targs, preds)
    return mauc, per, preds

# Cross-validation training
def run_cv(train_df, img_dir=IMG_DIR, n_folds=5, seed=42, epochs=18, batch_size=16, lr=3e-4, wd=1e-4, patience=5, backbone='tf_efficientnet_b3_ns', use_ema=True, device=device, mixup_alpha=0.3, oversample=True, ema_decay=0.9998, accum_steps=1):
    train_tfms, valid_tfms = get_transforms(IMG_SIZE)
    X = train_df[['image_id']].values
    y = train_df[TARGET_COLS].values
    mskf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    folds = list(mskf.split(X, y))
    # sanity check rare class distribution
    for fi, (tr_idx, va_idx) in enumerate(folds):
        cnt = train_df.iloc[va_idx]['multiple_diseases'].sum()
        print(f"Fold {fi}: val multiple_diseases count = {int(cnt)}", flush=True)
    oof = np.zeros((len(train_df), N_CLASSES), dtype=np.float32)
    per_fold_scores = []
    for fi, (tr_idx, va_idx) in enumerate(folds):
        print(f"\n=== Fold {fi+1}/{n_folds} ===", flush=True)
        t_start = time.time()
        tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
        va_df = train_df.iloc[va_idx].reset_index(drop=True)
        ds_tr = PlantDataset(tr_df, img_dir, img_col='image_id', target_cols=TARGET_COLS, transforms=train_tfms, is_train=True)
        ds_va = PlantDataset(va_df, img_dir, img_col='image_id', target_cols=TARGET_COLS, transforms=valid_tfms, is_train=True)
        # Oversampling for multiple_diseases class
        sampler = None
        if oversample:
            md = tr_df['multiple_diseases'].values.astype(np.float32)
            # weight boost for md positives
            w = np.ones(len(tr_df), dtype=np.float32)
            # inverse frequency style boost, capped at 10x
            pos_frac = md.mean() if md.mean() > 0 else 0.05
            boost = min(10.0, max(3.0, 1.0 / max(1e-3, pos_frac)))
            print(f"  Oversample: pos_frac={pos_frac:.4f}, boost={boost:.2f}", flush=True)
            w[md > 0.5] *= boost
            sampler = WeightedRandomSampler(weights=torch.from_numpy(w), num_samples=len(tr_df), replacement=True)
        dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=(sampler is None), sampler=sampler, num_workers=2, pin_memory=True, drop_last=True)
        dl_va = DataLoader(ds_va, batch_size=batch_size*2, shuffle=False, num_workers=2, pin_memory=True)
        model = Model(backbone=backbone, num_classes=N_CLASSES).to(device)
        optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)
        # OneCycleLR per batch
        steps_per_epoch = max(1, len(dl_tr))
        scheduler = OneCycleLR(optimizer, max_lr=lr, epochs=epochs, steps_per_epoch=steps_per_epoch, pct_start=0.3, anneal_strategy='cos', div_factor=25.0, final_div_factor=1e4)
        scaler = torch.cuda.amp.GradScaler(enabled=True)
        ema = EMA(model, decay=ema_decay) if use_ema else None
        loss_fn = AsymmetricLossMultiLabel(gamma_pos=0.0, gamma_neg=4.0, clip=0.05)
        best = -1.0
        best_state = None
        best_epoch = -1
        no_improve = 0
        for epoch in range(epochs):
            ep_s = time.time()
            tr_loss = train_one_epoch(model, ema, dl_tr, optimizer, scaler, loss_fn, device, scheduler=scheduler, mixup_alpha=mixup_alpha, accum_steps=accum_steps)
            mauc, per, preds = validate(model, dl_va, device, use_ema=True, ema=ema)
            elapsed = time.time()-ep_s
            print(f"Fold {fi} Epoch {epoch+1}/{epochs} | tr_loss {tr_loss:.4f} | val mAUC {mauc:.5f} | per {np.round(per,4)} | {elapsed:.1f}s", flush=True)
            if mauc > best:
                best = mauc; best_state = copy.deepcopy((ema.ema if use_ema else model)).state_dict(); best_epoch = epoch; no_improve = 0
            else:
                no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}", flush=True)
                break
        # load best and predict OOF
        (ema.ema if use_ema else model).load_state_dict(best_state, strict=True)
        mauc, per, preds = validate(model, dl_va, device, use_ema=True, ema=ema)
        oof[va_idx] = preds
        per_fold_scores.append((best, per))
        # save fold weights
        ckpt_path = f"model_fold{fi}.pth"
        torch.save(best_state, ckpt_path)
        print(f"Fold {fi} best mAUC {best:.5f} at epoch {best_epoch+1}. Saved {ckpt_path}. Fold time {time.time()-t_start:.1f}s", flush=True)
        del model, optimizer, scheduler, scaler, ema, dl_tr, dl_va, ds_tr, ds_va; gc.collect(); torch.cuda.empty_cache()
    # OOF score
    mauc, per = compute_auc(train_df[TARGET_COLS].values, oof)
    print(f"OOF mAUC: {mauc:.5f}; per-class: {np.round(per,5)}", flush=True)
    np.save('oof_preds.npy', oof)
    train_df[[*TARGET_COLS]] = train_df[TARGET_COLS]
    return oof, per_fold_scores

# Inference with TTA (orig + hflip + vflip + hvflip)
@torch.no_grad()
def predict_test(models_paths, test_df, img_dir=IMG_DIR, batch_size=32, backbone='tf_efficientnet_b3_ns', device=device):
    _, valid_tfms = get_transforms(IMG_SIZE)
    ds_te = PlantDataset(test_df, img_dir, img_col='image_id', target_cols=None, transforms=valid_tfms, is_train=False)
    dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    # prepare models
    models = []
    for p in models_paths:
        m = Model(backbone=backbone, num_classes=N_CLASSES).to(device)
        sd = torch.load(p, map_location=device)
        m.load_state_dict(sd, strict=True)
        m.eval()
        models.append(m)
    all_preds = []
    for images, ids in dl_te:
        images = images.to(device, non_blocking=True)
        logits_sum = None
        for m in models:
            logits0 = m(images)
            logits1 = m(torch.flip(images, dims=[3]))  # hflip
            logits2 = m(torch.flip(images, dims=[2]))  # vflip
            logits3 = m(torch.flip(images, dims=[2,3]))  # hvflip
            logits_tta = (logits0 + logits1 + logits2 + logits3) / 4.0
            logits_sum = logits_tta if logits_sum is None else logits_sum + logits_tta
        logits_avg = logits_sum / len(models)
        probs = torch.sigmoid(logits_avg).float().cpu().numpy()
        all_preds.append(probs)
    preds = np.concatenate(all_preds, axis=0)
    sub = pd.DataFrame(preds, columns=TARGET_COLS)
    sub.insert(0, 'image_id', test_df['image_id'].values)
    return sub

print('Pipeline defined. Ready to run CV training next.')

Device: cuda


Pipeline defined. Ready to run CV training next.


In [6]:
# Run 5-fold CV training and generate submission
import pandas as pd, numpy as np, time, os, gc

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# sanity: ensure columns exist
assert all([c in train_df.columns for c in TARGET_COLS])

SEED = 42
N_FOLDS = 5
EPOCHS = 18
BATCH_SIZE = 16
LR = 4e-4  # safer max_lr for OneCycle per expert audit
WD = 1e-4
PATIENCE = 5
BACKBONE = 'tf_efficientnet_b3_ns'
MIXUP_ALPHA = 0.3
EMA_DECAY = 0.999
OVERSAMPLE = True

t0 = time.time()
print('Starting CV with ASL + Mixup + Oversampling (cap 10x) + OneCycleLR + EMA...')
oof, per_fold_scores = run_cv(
    train_df, img_dir=IMG_DIR, n_folds=N_FOLDS, seed=SEED,
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR, wd=WD, patience=PATIENCE,
    backbone=BACKBONE, use_ema=True, mixup_alpha=MIXUP_ALPHA, oversample=OVERSAMPLE, ema_decay=EMA_DECAY, accum_steps=2
)
print(f'CV finished in {time.time()-t0:.1f}s')

# Collect fold checkpoints
ckpts = [f'model_fold{i}.pth' for i in range(N_FOLDS) if os.path.exists(f'model_fold{i}.pth')]
print('Found checkpoints:', ckpts)

print('Predicting test with 4-way TTA (orig, hflip, vflip, hvflip) and fold-ensemble...')
sub = predict_test(ckpts, test_df, img_dir=IMG_DIR, batch_size=32, backbone=BACKBONE)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape:', sub.shape)
sub.head()

Starting CV with ASL + Mixup + Oversampling (cap 10x) + OneCycleLR + EMA...
Fold 0: val multiple_diseases count = 17


Fold 1: val multiple_diseases count = 17


Fold 2: val multiple_diseases count = 17


Fold 3: val multiple_diseases count = 17


Fold 4: val multiple_diseases count = 17


...

Fold 4 best mAUC 0.97173 at epoch 18. Saved model_fold4.pth. Fold time 445.6s


OOF mAUC: 0.97510; per-class: [0.99578 0.91753 0.99608 0.991  ]


CV finished in 2262.1s
Found checkpoints: ['model_fold0.pth', 'model_fold1.pth', 'model_fold2.pth', 'model_fold3.pth', 'model_fold4.pth']
Predicting test with 4-way TTA (orig, hflip, vflip, hvflip) and fold-ensemble...


Saved submission.csv with shape: (183, 5)


,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.010960,0.212108,0.012376,0.986899
1,Test_1,0.220426,0.269574,0.010789,0.879246
2,Test_2,0.081844,0.155790,0.950597,0.067517
3,Test_3,0.043459,0.071360,0.994454,0.023010
4,Test_4,0.008794,0.041318,0.999413,0.010087
